### Going further

Now looking at the code, there is some code that is different than the other.  We can identify it by looking at our `run` method.

```python
def run(self):
    self._receipts_data = self.retrieve_receipts()
    self._receipts = self.receipts_data_to_objects(self._receipts_data)
    return self._receipts
```

We can see that this method does two things:
1. retrieve data from the api
2. coerce the data into Receipt objects.  

These are two tasks that we may not always want to perform together.  For example, we may want to retreive data from the API and create new restaurant objects, or new location objects.  So it makes sense to isolate this code.  Here's how.

1. We just move the `retrieve_receipts` method into a new class.

In [8]:
class TexasDrinksClient:
    def run(self):
        response = requests.get("https://data.texas.gov/resource/naix-2893.json?location_name=MAX%27S%20WINE%20DIVE")
        return response.json()

And then we remove the related code from the `ReceiptBuilder`, and replace it with a call to the `TexasDrinksClient`.

In [9]:
import requests
class ReceiptBuilder:
    def run(self):
        self._request_api = TexasDrinksClient()
        self._receipts_data = self._request_api.run()
        self._receipts = self.receipts_data_to_objects(self._receipts_data)
        return self._receipts
    
    # remove retrieve_receipts method
    
    def select_data(self, receipt_data):
        attributes = ('total_receipts', 'location_address', 'obligation_end_date_yyyymmdd', 'location_name')
        return dict((k, receipt_data[k]) for k in attributes)
    
    def create_receipt(self, receipt_data):
        receipt = Receipt(receipt_data['total_receipts'], receipt_data['location_address'], receipt_data['obligation_end_date_yyyymmdd'], receipt_data['location_name'])
        return receipt
    
    def receipts_data_to_objects(self, receipts_data):
        receipts = []
        for receipt_data in receipts_data:
            selected_data = self.select_data(receipt_data)
            receipt = self.create_receipt(selected_data)
            receipts.append(receipt)
        return receipts

And then we check to make sure that this still works.

In [10]:
builder = ReceiptBuilder()
receipts = builder.run()
receipts[0:3]

### Such small improvements?

It may seem silly to create a class for just one method.  Why do we?

In [22]:
class TexasDrinksAPI:
    def run(self):
        response = requests.get("https://data.texas.gov/resource/naix-2893.json?location_name=MAX%27S%20WINE%20DIVE")
        return response.json()

In part we do this because we are following a standard recipe.  If we left the method in the `ReceiptBuilder` class, no great sin would have been committed.

### Summary

In this lesson, we learned to remove code related to the API into it's own class.  The reason why we do this is because requesting the Texas API has value even when not related to building receipts.  And we want an easy tool to do this.

We also saw a standard pattern for reaching an API.  The main idea is that we do not like working with messy dictionaries and instead would prefer to work with objects.  We accomplish this by creating three classes: 

1. A `TexasDrinksAPI` class that makes requests to the API
2. A `ReceiptBuilder` class that turns the data into `Receipt` instances
3. The `Receipt` class